In [1]:
# Conducting initial exploration of the dataset given by Tiny Shakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#create a list of all the unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [2]:
#Next we want to tokenize the input i.e. Convert the characters into integers
#Create a mapping for characters to integers
s_to_i = { ch:i for i,ch in enumerate(chars) }
i_to_s = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [s_to_i[c] for c in s] #encoder: receive a string, ouput a list of integers
decode = lambda l: ''.join([i_to_s[i] for i in l]) #decoder: receive a list of integers, return a string

#we use this to encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

#split the data into train and validation sets (90-10 split)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(1337)
#define a block size to use i.e. Max content length
block_size = 8
#define a batch dimension i.e. Number of independent sequences processed in parallel
batch_size = 4

def get_batch(split):
    #generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    #generates random offsets to find subsets of the data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    #y represents the target values, as they are the x subsets offset by 1
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [15]:
#to visualise the get_batch function and the independant examples
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 6,  0, 21, 44,  1, 61, 43,  1],
        [58, 52, 43, 57, 57,  2,  1, 57],
        [ 1, 59, 52, 39, 41, 46, 47, 52],
        [43, 42, 50, 39, 56,  6,  1, 50]])
targets:
torch.Size([4, 8])
tensor([[ 0, 21, 44,  1, 61, 43,  1, 61],
        [52, 43, 57, 57,  2,  1, 57, 43],
        [59, 52, 39, 41, 46, 47, 52, 45],
        [42, 50, 39, 56,  6,  1, 50, 43]])
----
When input is [6] the target: 0
When input is [6, 0] the target: 21
When input is [6, 0, 21] the target: 44
When input is [6, 0, 21, 44] the target: 1
When input is [6, 0, 21, 44, 1] the target: 61
When input is [6, 0, 21, 44, 1, 61] the target: 43
When input is [6, 0, 21, 44, 1, 61, 43] the target: 1
When input is [6, 0, 21, 44, 1, 61, 43, 1] the target: 61
When input is [58] the target: 52
When input is [58, 52] the target: 43
When input is [58, 52, 43] the target: 57
When input is [58, 52, 43, 57] the target: 57
When input is [58, 52, 43, 57, 57] the target: 2
When input is [58, 52, 43, 57, 5

In [25]:
#we will try feeding this data into a Bigram language model, the simplest of language neural networks
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    #using this method, we are predicting the next character based on a single token (no context)
    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C)
        #logits are the scores for the next character in the sequence

        if targets is None:
            loss = None
        else:
            #reshape logits to find loss
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #evaluate the loss of each character using negative log likelihood
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes (B, C), as we are only focusing on the last character in the block
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) #(B,C)
            #get the best sample from the probability distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
#now we have scores for each character in the dataset above, for each character in the vocabularyj
print(logits.shape)
print(loss)

#test the generation
#in our data, 0 represents a newline, so we start the generation with a 0
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7051, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3
